In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.61 Safari/537.36"}
url = "https://www.aladin.co.kr/search/wsearchresult.aspx?SearchTarget=Book&SearchWord="

input_path = 'test/test.xls'

Load excel file and remove top and bottom

In [2]:
df = pd.read_excel(input_path, header=None, sheet_name=0)
df = df[12:-1].reset_index(drop=True)
df.columns = df.loc[0]
df = df[1:]
df['상태'] = ''
df.head(10)

,NO,도서명,저자,출판사,정가,수량,금액,납품가,상태
1,1,105개의 수도로 만나는 세계,박동석,책숲,15000,1,15000,13500,
2,2,10만 원이 10억 되는 재밌는 돈 공부,제임스 맥케나,앵글북스,15000,1,15000,13500,
3,3,13개월 13주 13일 도둑맞은 시간,알렉스 쉬어러,책과콩나무,13000,1,13000,11700,
4,4,13일의 단톡방,방미진,상상의집,10800,1,10800,9720,
5,5,143층 나무집,앤디 그리피스,시공주니어,12000,1,12000,10800,
6,6,"3.1운동, 그 가족에게 생긴 일",고수산나,내일을여는책,11000,1,11000,9900,
7,7,35 : 그 전에 꼭 알아야 할 두 나라 역사,이윤희,하마,15000,1,15000,13500,
8,8,"46억 년의 비밀, 생명을 살리는 지구",김태영,뭉치,13000,1,13000,11700,
9,9,63일,허정윤,반달(킨더랜드),15000,1,15000,13500,
10,10,가난한 아이들의 선생님,파브리치오 실레이,지양사,12500,1,12500,11250,


Test URL

In [10]:
res = requests.get(url, headers)
print(f'request status : {res.status_code}')

request status : 200


In [40]:
def remove_space(title):
    return title.replace(" ", "")

def search_by_book_name(title, company):
    output = []
    _title = remove_space(title)
    
    res = requests.get(url + title)
    if res.status_code == 200:
        html = res.text
        soup = BeautifulSoup(html, 'html.parser')
        
        search_result = soup.find("div", attrs={"id":"Search3_Result"})
        if search_result is None:
            return output
        
        books = search_result.find_all("div", attrs={"class":"ss_book_box"})
        if books is None:
            return output

        is_find = False     
        for book in books:
            if is_find == True:     
                break
            b_status = book.find("div", attrs={"class" : "book_Rfloat_02"}).find("div").get_text()
            b_list = book.find("div", attrs={"class" : "ss_book_list"}).find_all("li")
            
            b_price = ""
            b_title = ""
            b_company = ""
            b_authors = []
           
            for idx in range(len(b_list)):
                sale_price = b_list[idx].find("span", attrs={"class" : "ss_p2"}) # 할인가
                if sale_price != None:
                    price = b_list[idx].find("span") # 정가
                    b_price = price.get_text()
                                        
                info = b_list[idx].find("a", attrs={"class" : "bo3"})
                if info != None:
                    b_title = info.get_text()
                    
                    detail = b_list[idx + 1].find_all("a")
                    b_authors = [author.get_text() for author in detail[:-1]]
                    b_company = detail[-1].get_text()
            _b_title = remove_space(b_title)

            if _b_title == _title and b_company == company:
                print("검색 결과 : "+_b_title, b_authors, b_company, b_price, b_status)
                output.append([_b_title, b_authors, b_company, b_price, b_status])
                is_find = True
                
        return output

In [62]:
for idx in range(1, 6):
    data = df.loc[idx]
    title = data['도서명']
    company = data['출판사']
    try:
        print('검색 도서명: {} 출판사: {}'.format(title, company))
        output = search_by_book_name(title, company)
        if len(output) == 0:
            df.loc[idx, '상태'] = '검색불가'
            continue
        if len(output) == 1:
            output = output[0]
        if len(output) > 1:
            for i in output:
                if i[0] == data['도서명'] and i[2] == data['출판사']:
                    output = i

        b_title = output[0]
        b_company = output[2]
        b_status = output[4]
        b_price = ''    
        for p in output[3].split(','): # 금액에 쉼표 제거
            b_price += p
        if b_price != str(data['정가']):
            df.loc[idx, '상태'] = "가격불일치:"+b_price
        if b_status == '장바구니':
            df.loc[idx, '상태'] = 'O'
        if b_status != '장바구니':
            df.loc[idx, '상태'] = b_status
    
    except:
        print('error')

df.head()


검색 도서명: 애착교실 출판사: 해냄
검색 결과 : 애착교실 ['루이스 코졸리노', '서영조', '최성애'] 해냄 16,000 장바구니
검색 도서명: 애착 수업 출판사: 푸른숲
검색 결과 : 애착수업 ['오카다 다카시', '이정환'] 푸른숲 15,000 장바구니
검색 도서명: 어서 오세요, 휴남동 서점입니다 출판사: 클레이하우스
검색 도서명: 세븐 테크 출판사: 웅진지식하우스
검색 결과 : 세븐테크 ['김미경', '김상균', '김세규', '김승주', '이경전', '이한주', '정지훈', '최재붕', '한재권'] 웅진지식하우스 18,000 장바구니
검색 도서명: 마음챙김 출판사: 안드로메디안
검색 결과 : 마음챙김 ['샤우나 샤피로', '박미경'] 안드로메디안 16,500 장바구니


,NO,도서명,지은이,출판사,정가,수량,금액,납품가,상태
1,1,애착교실,루이스 코졸리노,해냄,16000,1,16000,14400,O
2,2,애착 수업,오카다 다카시,푸른숲,15000,1,15000,13500,O
3,3,"어서 오세요, 휴남동 서점입니다",황보름,클레이하우스,15000,1,15000,13500,검색불가
4,4,세븐 테크,김미경 외,웅진지식하우스,18000,1,18000,16200,O
5,5,마음챙김,샤우나 샤피로,안드로메디안,16500,1,16500,14850,O
